## Text2SQL using Amazon Bedrock LLM Models

This notebook generates SQL statement and summary of input query based on the given LLM and input question. It also uses tools to map a text like gender, race and state of patients into their corresponding code.

The database is expected to be in Amazon Redshift. Each section contains a single database starting with `DESynPUF`.

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import jupyter_black

jupyter_black.load()

In [4]:
import pandas as pd
import os
import boto3

from tqdm import tqdm

tqdm.pandas()

from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

from langchain.embeddings import BedrockEmbeddings

from text2sql.conf.config import *
from text2sql.lib.utils import *
from text2sql.lib.function_calls import *

## 1. DESynPUF Database

### 1.1 Run LLM with Sample Questions

In [5]:
DB_NAME = "DESynPUF"
SAMPLE_QUESTIONS_PATH = "../data/databases/synpuf/sample_questions.txt"
ALL_QA_PATH = "s3://aws-gaiic-merck-text2sql/data/synpuf_query_combined_20231121.csv"
OUTPUT_DIR = "../data/databases/synpuf/outputs/"

cfg_synpuf = DATABASES[DB_NAME]
TABLES_DESC_PATH = cfg_synpuf["tables_desc_path"]
COLS_DESC_PATH = cfg_synpuf["columns_desc_path"]
REDSHIFT_PARAMS = cfg_synpuf["redshift_params"]
SQL_MODEL_ID = "anthropic.claude-v2"
EMBED_MODEL_ID = "amazon.titan-embed-g1-text-02"

# Whether to count input and completion tokens
COUNT_TOKENS = True

In [6]:
print(json.dumps(cfg_synpuf, indent=4))

{
    "tables_desc_path": "../data/databases/synpuf/tables_description.csv",
    "columns_desc_path": "../data/databases/synpuf/columns_description.csv",
    "tables_info_path": "../data/databases/synpuf/db_tables_info.txt",
    "few_shot": {
        "vectordb_dir": "../data/databases/synpuf/faiss_index/",
        "golden_examples_path": "../data/databases/synpuf/few_shot_examples.json",
        "num_few_shot_examples": 3
    },
    "redshift_params": {
        "schema_name": "rwdex_raw_synpuf",
        "host": "redshift-synpuf-db-cluster-1.ckbiikj73zqo.us-east-1.redshift.amazonaws.com",
        "port": 5439,
        "database": "dev",
        "username": "awsuser",
        "password": "Merck123"
    }
}


In [7]:
with open(SAMPLE_QUESTIONS_PATH, "r") as fp:
    sample_questions = fp.readlines()
sample_questions = [qn.strip() for qn in sample_questions]

sample_questions

['Find total number of inpatients',
 'Select all inpatients in year 2005',
 'Count female patients',
 'Find total number of male inpatients',
 'Find total number of female, black outpatients',
 'Find total number of female, white patients living in California',
 'Select the outpatients who are diagnosed with diabetes of code list X',
 'Select the male inpatients who receive hospitalization because of diabetes of code list X',
 'Select the male, white outpatients living in Florida who are diagnosed with disease in code list X and are sent to hospital thereafter']

In [8]:
# Get table descriptions in xml format
df = pd.read_csv(TABLES_DESC_PATH)
table_descriptions = df.to_markdown(index=False)
print(table_descriptions[:2000])
print("...")

| table_name               | table_description                               |
|:-------------------------|:------------------------------------------------|
| beneficiary_summary      | It contains synthesized Medicaire beneficiaries |
| inpatient_claims         | It contains information about inpatient claims  |
| outpatient_claims        | It stores data about outpatient claims          |
| prescription_drug_events | Contains prescription drug events               |
| carrier_claims           | Stores carrier claims                           |
...


In [9]:
# Get column descriptions in xml format
df = pd.read_csv(COLS_DESC_PATH)
col_descriptions = df.to_markdown(index=False)
print(col_descriptions[:2000])
print("...")

| column_name                       | column_description                                                  | table_name               |
|:----------------------------------|:--------------------------------------------------------------------|:-------------------------|
| DESYNPUF_ID                       | Beneficiary Code                                                    | beneficiary_summary      |
| BENE_BIRTH_DT                     | Date of birth                                                       | beneficiary_summary      |
| BENE_DEATH_DT                     | Date of death                                                       | beneficiary_summary      |
| BENE_SEX_IDENT_CD                 | Sex                                                                 | beneficiary_summary      |
| BENE_RACE_CD                      | Beneficiary Race Code                                               | beneficiary_summary      |
| BENE_ESRD_IND                     | End stage renal d

In [11]:
# Get tables info with sample rows
tables_info = get_db_tables_info(REDSHIFT_PARAMS, cfg_synpuf["tables_info_path"])
print(tables_info[:200])
print("...")

Tables Info already exists! Loading...


CREATE TABLE rwdex_raw_synpuf.beneficiary_summary (
	desynpuf_id VARCHAR(256), 
	bene_birth_dt DATE, 
	bene_death_dt DATE, 
	bene_sex_ident_cd INTEGER, 
	bene_race_cd INTEGER, 
	bene_esrd_ind VARCHAR
...


In [12]:
# Create LLM for SQL generation
llm = get_bedrock_llm(
    model_id=SQL_MODEL_ID, model_config=SQL_MODEL_CONFIG, verbose=False
)

In [13]:
print(PROMPT)



Human: You are a PostgreSQL expert. Given an input question, first create a syntactically correct PostgreSQL query to run, then look at the results of the query and return the answer to the input question.
Never query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in double quotes (") to denote them as delimited identifiers.
Pay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table. Database schema is provided in <table-info> tag and input question is in <question> tag. All tables description is provided within <tables-description> tag. Columns description is also provided within <columns-description> tag. In addition, the database schema name is also added in <schema-name> tag.

Provide only the necessary answer. When count, make sure you count only the distinct ones. When using any

In [17]:
for question in sample_questions[:2]:
    print("Input Question:\n", question)
    print("-" * 40)
    # Find few-shot examples if needed
    few_shot_examples = ""
    if cfg_synpuf["few_shot"]["num_few_shot_examples"]:
        # Create embeddings model for few-shot learning
        bedrock_runtime = boto3.client("bedrock-runtime")
        llm_embeddings = BedrockEmbeddings(
            client=bedrock_runtime, model_id=EMBED_MODEL_ID
        )

        # Load golden examples for few shot learning
        with open(cfg_synpuf["few_shot"]["golden_examples_path"], "r") as fp:
            golden_examples = json.load(fp)

        # # Show some golden examples
        # i = 0
        # for key, val in golden_examples.items():
        #     print(f"Question: {key}\nSQL: {val}")
        #     print("-" * 30)
        #     i += 1
        #     if i == 3:
        #         break

        # Load Vector DB and Create a retriever (create first if not existing)
        retriever = get_vertordb_retriever(
            llm_embeddings,
            cfg_synpuf["few_shot"]["vectordb_dir"],
            cfg_synpuf["few_shot"]["num_few_shot_examples"],
            golden_questions=list(golden_examples.keys()),
        )
        # vector_db = FAISS.load_local(cfg_synpuf['few_shot']["vectordb_dir"], llm_embeddings)
        # retriever = vector_db.as_retriever(search_kwargs={"k": cfg_synpuf['few_shot']['num_few_shot_examples']})

        few_shot_examples = build_few_shot_examples(
            golden_examples, question, retriever
        )

    inputs = {
        "question": question,
        "db_schema_name": REDSHIFT_PARAMS["schema_name"],
        "table_info": tables_info,
        "tables_description": table_descriptions,
        "columns_description": col_descriptions,
        "query_examples": few_shot_examples,
        "history": "",
    }

    prompt = create_prompt(PROMPT, inputs)
    response = run_loop(llm, prompt, count_tokens=COUNT_TOKENS)

    sql = extract_xml(tags=["sql"], text=response)
    summary = extract_xml(tags=["summary"], text=response)

    print("SQL:\n", sql)
    if sql is None:
        print("Response:\n", response)
        print("-" * 40)
    print("-" * 40)
    print("Question Summary:\n", summary)
    print("=" * 40)

Input Question:
 Find total number of inpatients
----------------------------------------
Prompt Total Tokens: 8008
Response Total Tokens: 108
SQL:
 SELECT COUNT(DISTINCT ic.desynpuf_id) AS num_inpatients
FROM rwdex_raw_synpuf.inpatient_claims ic
----------------------------------------
Question Summary:
 The SQL query counts the distinct desynpuf_id values from the inpatient_claims table to find the total number of inpatients. It uses COUNT(DISTINCT) to eliminate duplicate desynpuf_id values and ensure only unique inpatients are counted.
Input Question:
 Select all inpatients in year 2005
----------------------------------------
Prompt Total Tokens: 8009
Response Total Tokens: 46
Prompt Total Tokens: 8107
Response Total Tokens: 156
SQL:
 SELECT DISTINCT ic.desynpuf_id
FROM rwdex_raw_synpuf.inpatient_claims ic
INNER JOIN rwdex_raw_synpuf.beneficiary_summary bs ON ic.desynpuf_id = bs.desynpuf_id
WHERE ic.clm_from_dt BETWEEN '2005-01-01' AND '2005-12-31'
---------------------------------

### 1.2 Run for all examples

In [ ]:
# Load input data
df_qa = pd.read_csv(ALL_QA_PATH)

# Remove all gpt4 columns
gpt4_cols = [col for col in df_qa.columns if "gpt4" in col]
df_qa.drop(columns=gpt4_cols, inplace=True)
print(df_qa.shape)
df_qa.head()

In [ ]:
# Count number of executable SQLs
df_qa.query_corrected_with_list_executed.value_counts()

In [ ]:
inputs = {
    "question": "<<question here>>",
    "db_schema_name": REDSHIFT_PARAMS["schema_name"],
    "table_info": tables_info,
    "tables_description": table_descriptions,
    "columns_description": col_descriptions,
    "query_examples": few_shot_examples,
    "history": "",
}

gt_sql_executed_col = "query_corrected_with_list_executed"
question_col = "question_with_list"

df_qa0 = df_qa.iloc[:5].progress_apply(
    get_llm_output,
    args=(llm, PROMPT, inputs, question_col, gt_sql_executed_col),
    axis=1,
)

In [ ]:
df_qa0.head()

In [ ]:
# Display GT and predicted SQL
_ = df_qa0.apply(
    print_gt_and_pred_sql,
    args=("question", "query_corrected", "sql_predicted", "question_summary"),
    axis=1,
)

In [ ]:
from datetime import datetime

## Save Outputs
os.makedirs(OUTPUT_DIR, exist_ok=True)
now = datetime.now().strftime("%Y%m%d-%H%M%S")
output_path = os.path.join(OUTPUT_DIR, f"llm-predictions-{now}.csv")
df_qa.to_csv(output_path, index=False)

## 2. Premier Hospital Database

### 2.1 Run LLM with Sample Questions

In [22]:
DB_NAME = "Premier_Hospital_Database"
SAMPLE_QUESTIONS_PATH = "../data/databases/premier/sample_questions.txt"
OUTPUT_DIR = "../data/databases/premier/outputs/"

cfg_premier = DATABASES[DB_NAME]
TABLES_DESC_PATH = cfg_premier["tables_desc_path"]
COLS_DESC_PATH = cfg_premier["columns_desc_path"]
REDSHIFT_PARAMS = cfg_premier["redshift_params"]
SQL_MODEL_ID = "anthropic.claude-v2"
EMBED_MODEL_ID = "amazon.titan-embed-g1-text-02"

In [23]:
print(json.dumps(cfg_premier, indent=4))

{
    "tables_desc_path": "../data/databases/premier/tables_description.csv",
    "columns_desc_path": "../data/databases/premier/columns_description.csv",
    "tables_info_path": "../data/databases/premier/db_tables_info.txt",
    "few_shot": {
        "vectordb_dir": "../data/databases/premier/faiss_index/",
        "golden_examples_path": "../data/databases/premier/few_shot_examples.json",
        "num_few_shot_examples": 0
    },
    "redshift_params": {
        "schema_name": "rwdex_raw_premier_hospital",
        "host": "redshift-premier-db-cluster-1.ckbiikj73zqo.us-east-1.redshift.amazonaws.com",
        "port": 5439,
        "database": "dev",
        "username": "awsuser",
        "password": "Merck123"
    }
}


In [24]:
with open(SAMPLE_QUESTIONS_PATH, "r") as fp:
    sample_questions = fp.readlines()
sample_questions = [qn.strip() for qn in sample_questions]

sample_questions

['Find total number of all the procedures']

In [25]:
# Get table descriptions in xml format
df = pd.read_csv(TABLES_DESC_PATH)
table_descriptions = df.to_markdown(index=False)
print(table_descriptions[:2000])
print("...")

| table_name             | table_description                                                                |
|:-----------------------|:---------------------------------------------------------------------------------|
| PATDEMO                | Patient demographic and hospital encounter characteristics                       |
| PATAPRDRG ***          | 3M™ APR-DRG classification for inpatient hospital encounters                     |
| PATBILL                | Detail charges per patient                                                       |
| PATCPT                 | Hospital submitted CPT® and HCPCS codes for each hospital encounter              |
| PATICD_DIAG            | ICD Diagnosis Codes submitted for each hospital encounter                        |
| PATICD_PROC            | ICD Procedure Codes submitted for each hospital encounter                        |
| ADMTYPE                | Admission Type codes and descriptions                                            |
| APRDRG *

In [26]:
# Get column descriptions in xml format
df = pd.read_csv(COLS_DESC_PATH)
col_descriptions = df.to_markdown(index=False)
print(col_descriptions[:2000])
print("...")

| column_name                  | column_description                                                                                                                                                                                             | table_name         | valid_values_or_link to_lookup_tab          |
|:-----------------------------|:---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|:-------------------|:--------------------------------------------|
| pat_key                      | Unique hospital encounter identifier (de-identified)                                                                                                                                                           | PATDEMO            | nan                                         |
| medrec_key                   | Unique patient identifier (de-identified

In [28]:
# Get tables info with sample rows
tables_info = get_db_tables_info(REDSHIFT_PARAMS, cfg_synpuf["tables_info_path"])
print(tables_info[:200])
print("...")

Tables Info already exists! Loading...


CREATE TABLE rwdex_raw_synpuf.beneficiary_summary (
	desynpuf_id VARCHAR(256), 
	bene_birth_dt DATE, 
	bene_death_dt DATE, 
	bene_sex_ident_cd INTEGER, 
	bene_race_cd INTEGER, 
	bene_esrd_ind VARCHAR
...


In [29]:
print(PROMPT[:1000] + "\n......")



Human: You are a PostgreSQL expert. Given an input question, first create a syntactically correct PostgreSQL query to run, then look at the results of the query and return the answer to the input question.
Never query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in double quotes (") to denote them as delimited identifiers.
Pay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table. Database schema is provided in <table-info> tag and input question is in <question> tag. All tables description is provided within <tables-description> tag. Columns description is also provided within <columns-description> tag. In addition, the database schema name is also added in <schema-name> tag.

Provide only the necessary answer. When count, make sure you count only the distinct ones. When using any

In [30]:
# Create LLM for SQL generation
llm = get_bedrock_llm(
    model_id=SQL_MODEL_ID, model_config=SQL_MODEL_CONFIG, verbose=False
)

In [32]:
for question in sample_questions[:2]:
    # Find few-shot examples if needed
    few_shot_examples = ""
    if cfg_synpuf["few_shot"]["num_few_shot_examples"]:
        # Create embeddings model for few-shot learning
        bedrock_runtime = boto3.client("bedrock-runtime")
        llm_embeddings = BedrockEmbeddings(
            client=bedrock_runtime, model_id=EMBED_MODEL_ID
        )

        # Load golden examples for few shot learning
        with open(cfg_synpuf["few_shot"]["golden_examples_path"], "r") as fp:
            golden_examples = json.load(fp)

        # # Show some golden examples
        # i = 0
        # for key, val in golden_examples.items():
        #     print(f"Question: {key}\nSQL: {val}")
        #     print("-" * 30)
        #     i += 1
        #     if i == 3:
        #         break

        # Load Vector DB and Create a retriever (create first if not existing)
        retriever = get_vertordb_retriever(
            llm_embeddings,
            cfg_synpuf["few_shot"]["vectordb_dir"],
            cfg_synpuf["few_shot"]["num_few_shot_examples"],
            golden_questions=list(golden_examples.keys()),
        )
        # vector_db = FAISS.load_local(cfg_synpuf['few_shot']["vectordb_dir"], llm_embeddings)
        # retriever = vector_db.as_retriever(search_kwargs={"k": cfg_synpuf['few_shot']['num_few_shot_examples']})

        few_shot_examples = build_few_shot_examples(
            golden_examples, question, retriever
        )

    inputs = {
        "question": question,
        "db_schema_name": REDSHIFT_PARAMS["schema_name"],
        "table_info": tables_info,
        "tables_description": table_descriptions,
        "columns_description": col_descriptions,
        "query_examples": few_shot_examples,
        "history": "",
    }

    prompt = create_prompt(PROMPT, inputs)
    response = run_loop(llm, prompt, count_tokens=COUNT_TOKENS)

    sql = extract_xml(tags=["sql"], text=response)
    summary = extract_xml(tags=["summary"], text=response)

    print("Input Question:\n", question)
    print("-" * 40)
    print("SQL:\n", sql)
    if sql is None:
        print("Response:\n", response)
        print("-" * 40)
    print("-" * 40)
    print("Question Summary:\n", summary)
    print("=" * 40)

Prompt Total Tokens: 40327
Response Total Tokens: 79
Input Question:
 Find total number of all the procedures
----------------------------------------
SQL:
 SELECT COUNT(DISTINCT icd_code) AS total_procedures
FROM rwdex_raw_synpuf.paticd_proc
----------------------------------------
Question Summary:
 This query selects the distinct ICD procedure codes from the PATICD_PROC table and counts them to get the total number of procedures.


### 2.2 Run for all examples

In [ ]:
# Load input data
df_qa = pd.read_csv(ALL_QA_PATH)

# Remove all gpt4 columns
gpt4_cols = [col for col in df_qa.columns if "gpt4" in col]
df_qa.drop(columns=gpt4_cols, inplace=True)
print(df_qa.shape)
df_qa.head()

In [ ]:
# Count number of executable SQLs
df_qa.query_corrected_with_list_executed.value_counts()

In [ ]:
inputs = {
    "question": "<<question here>>",
    "db_schema_name": REDSHIFT_PARAMS["schema_name"],
    "table_info": tables_info,
    "tables_description": table_descriptions,
    "columns_description": col_descriptions,
    "query_examples": few_shot_examples,
    "history": "",
}

gt_sql_executed_col = "query_corrected_with_list_executed"
question_col = "question_with_list"

df_qa0 = df_qa.iloc[:5].progress_apply(
    get_llm_output,
    args=(llm, PROMPT, inputs, question_col, gt_sql_executed_col),
    axis=1,
)

In [ ]:
df_qa0.head()

In [ ]:
# Display GT and predicted SQL
_ = df_qa0.apply(
    print_gt_and_pred_sql,
    args=("question", "query_corrected", "sql_predicted", "question_summary"),
    axis=1,
)

In [ ]:
from datetime import datetime

## Save Outputs
os.makedirs(OUTPUT_DIR, exist_ok=True)
now = datetime.now().strftime("%Y%m%d-%H%M%S")
output_path = os.path.join(OUTPUT_DIR, f"llm-predictions-{now}.csv")
df_qa.to_csv(output_path, index=False)